### Data Ingestion and Data Preprocessing

In [4]:
import requests
from telethon import TelegramClient
import pandas as pd
import re
import nest_asyncio
import asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Telegram API configuration
api_id = 28162665
api_hash = 'd5427ba01ae31a3a1c876f41e7e11d42'
phone_number = '+251977951226'

# Initialize the Telegram client
client = TelegramClient('session_name', api_id, api_hash)

# List of e-commerce channels
channels = [
    '@ZemenExpress', '@sinayelj', '@MerttEka', '@yebatochmregagroup', '@helloomarketethiopia',
    '@Leyueqa', '@kstoreaddis', '@Fashiontera'
]

# Connect and fetch messages from a single channel
async def fetch_telegram_messages(channel_name, limit=100):
    async for message in client.iter_messages(channel_name, limit=limit):
        if message.message:
            yield {
                'sender': message.sender_id,
                'timestamp': message.date,
                'channel': channel_name,  # Add the channel name to the message data
                'message': message.message
            }

# Clean and preprocess Amharic messages
def clean_amharic_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text.strip()

# Fetch messages from all channels
async def fetch_from_multiple_channels(channels, limit=100):
    all_messages = []
    for channel in channels:
        print(f"Fetching messages from {channel}...")
        async for msg in fetch_telegram_messages(channel, limit=limit):
            all_messages.append(msg)  # Add the messages from each channel to the main list
    return all_messages

# Main function to run the code
async def main():
    # Connect to Telegram using async with
    async with client:
        # Fetch messages from all channels
        all_messages = await fetch_from_multiple_channels(channels, limit=500)
        
        # Create DataFrame and clean messages
        df = pd.DataFrame(all_messages)
        df['clean_message'] = df['message'].apply(clean_amharic_text)
        
        # Save preprocessed data to CSV
        df.to_csv('preprocessed_telegram_data.csv', index=False)
        print("Messages have been saved to preprocessed_telegram_data.csv")

# Run the main async function
await main()


Fetching messages from @ZemenExpress...
Fetching messages from @sinayelj...
Fetching messages from @MerttEka...
Fetching messages from @yebatochmregagroup...
Fetching messages from @helloomarketethiopia...
Fetching messages from @Leyueqa...
Fetching messages from @kstoreaddis...
Fetching messages from @Fashiontera...
Messages have been saved to preprocessed_telegram_data.csv
